In [1]:
#importing scikit libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
    

In [2]:
#pca function
def pcafunction(x_data, y_data, num_componentes, name_column, name_index):

    # standarizar
    scaler_model = StandardScaler().fit(x_data)
    x = scaler_model.transform(x_data) 
 
       
    pca = PCA(n_components=num_componentes)
    principalComponents = pca.fit_transform(x)
    
    pca_df = pd.DataFrame(principalComponents, columns=[f'PC1_{name_column}',
                                                        f'PC2_{name_column}'
                                                       ])
    pca_df['Date_timed'] = y_data    
    
    return pca,pca_df,scaler_model #Scaler_model parameters

In [3]:
def exogen_pca(dataframe, var_index):
   
    
    dfvar = dataframe[['Energy_Demand', 'Day', 'Hour', 'Month','Tmax-Cab', 'Tmax-Hmo',
               'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
               'Tmin-Lmo', 'Tmin-Cul','Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm',
               'Prec_Cul_Mm',
               'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
               'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
               'Christmas_NY', 'After_Christmas_NY']]
    
    dfvar['Date_timed']= dfvar.index
    
    #variables para pca
    y = dataframe.loc[:,[var_index]].values
    
    #PCA CLIMATOLOGICAS
    x_climatologicas = dataframe.loc[:,'Tmax-Cab':'Prec_Cul_Mm'].values 
    pca_clima,pca_df_clima, scaler_model_clima = pcafunction(x_climatologicas, y, 2, 'Weather', var_index)
    
    
    #UNION DE LOS 2 DATAFRAMES
    pca_df = pd.merge(pca_df_clima, dfvar, left_on=var_index, right_on='Date_timed', how='left')
    del pca_df['Date_timed']
    
    return pca_df,pca_clima,scaler_model_clima

In [4]:
def new_dataframe(my_array,y_data,name_column, name_index ):
    df = pd.DataFrame(my_array, columns=[f'PC1_{name_column}', f'PC2_{name_column}'])
    df['Date_timed'] = y_data
    
    return df

In [5]:
def transform_data(dataframe, var_index,pca_clima,scaler_model_clima):
    
    #variables
    dfvar = dataframe[['Energy_Demand', 'Day', 'Hour', 'Month', 'Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul', 'Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm',
       'Prec_Cul_Mm',
        'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
       'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
       'Christmas_NY', 'After_Christmas_NY']]
    dfvar['Date_timed']= dfvar.index
    #variables y
    y = dfvar.index
    
    ##### PCA CLIMA #####
    x_clima = dataframe.loc[:,'Tmax-Cab':'Prec_Cul_Mm'].values 
    #standarizacion  CLIMA
    x_df_clima = scaler_model_clima.transform(x_clima) 
    #trasnformacion
    clima = pca_clima.transform(x_df_clima)
    
    df_clima = new_dataframe(clima,y,'Weather', var_index )
    
    #UNION DE DATAFRAMES
    pca_df = pd.merge(df_clima, dfvar, left_on=var_index, right_on='Date_timed', how='left')
    
    return pca_df
    